Google satellite images might not seem too spectacular at first sight, but actually it is a masterpiece. It is not a single image, but a composition of smaller images:
- from several data providers
- from several points in time
- with different resolutions

Overall, we get one large image that has very high resolution in many parts of the world (big cities). And also with almost all clouds removed.

Here, we want to compare the resolution in New York with the resolution of Sentinel 2.

In [ ]:
import geemap
import ee
from egis.utils import (
    vis_params_dict, extract_single_img
)

In [ ]:
ee.Initialize()

### Select interesting areas

In [ ]:
coord_list = [[-74.021956, 40.699491],
              [-73.998517, 40.699491],
              [-73.998517, 40.714391],
              [-74.021956, 40.714391],
              [-74.021956, 40.699491]]
roi_clip_region = ee.Geometry.Polygon(coord_list)

# compute center point
roi_center_point = roi_clip_region.centroid()

# extract coordinates for centering of maps
roi_center_coords = roi_center_point.getInfo()['coordinates']
roi_center_coords = [roi_center_coords[1], roi_center_coords[0]]

In [ ]:
google_map = geemap.Map(center=roi_center_coords, zoom=16)
google_map.add_basemap('SATELLITE')
google_map

### Sentinel image

In [ ]:
sentinel_image = ee.Image('COPERNICUS/S2_SR_HARMONIZED/20221027T155341_20221027T155837_T18TWL')

In [ ]:
vis_params = vis_params_dict(bands=['B4','B3','B2'],
                                     min=0,
                                     max=3000,
                                     gamma=1.4)

In [ ]:
sentinel_map = geemap.Map()
sentinel_map.addLayer(sentinel_image, vis_params)
sentinel_map.centerObject(roi_center_point, 16)
sentinel_map